In [2]:
# from fund.fund import Fund
# from fund.states import States
from strats.speculation import Speculation
from modeler_strats.modeler_strat_fact import ModelerStratFactory as modelstratfact
import pickle

In [3]:
speculation = Speculation(4)

In [4]:
start_year = 2023
end_year = 2024

In [5]:
model_strats = [
                "universal",
                "industry",
               ]

In [6]:
speculation.pull_included_columns()
speculation.pull_factors()

In [7]:
data = speculation.pull_training_data()

In [8]:
data[data["year"]==2022]

,year,week,ticker,GICS Sector,GICS Sub-Industry,adjclose,y,0,1,2,...,4,5,6,7,8,9,10,11,12,13
509,2022,1,MMM,Industrials,Industrial Conglomerates,171.729747,171.760433,171.729747,170.312427,166.177936,...,169.317042,164.909732,170.756898,173.282065,172.935151,172.370214,171.449578,172.975096,169.526512,168.733319
510,2022,2,MMM,Industrials,Industrial Conglomerates,171.760433,168.529748,171.760433,171.729747,170.312427,...,168.417552,169.317042,164.909732,170.756898,173.282065,172.935151,172.370214,171.449578,172.975096,169.526512
511,2022,3,MMM,Industrials,Industrial Conglomerates,168.529748,162.849919,168.529748,171.760433,171.729747,...,166.177936,168.417552,169.317042,164.909732,170.756898,173.282065,172.935151,172.370214,171.449578,172.975096
512,2022,4,MMM,Industrials,Industrial Conglomerates,162.849919,158.269998,162.849919,168.529748,171.760433,...,170.312427,166.177936,168.417552,169.317042,164.909732,170.756898,173.282065,172.935151,172.370214,171.449578
513,2022,5,MMM,Industrials,Industrial Conglomerates,158.269998,154.677791,158.269998,162.849919,168.529748,...,171.729747,170.312427,166.177936,168.417552,169.317042,164.909732,170.756898,173.282065,172.935151,172.370214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44664,2022,48,ADP,Information Technology,Data Processing & Outsourced Services,262.215591,259.129684,262.215591,260.273924,252.106184,...,238.003970,236.347930,231.598759,225.890198,232.866664,227.337243,230.975753,234.441096,239.858583,242.451385
44665,2022,49,ADP,Information Technology,Data Processing & Outsourced Services,259.129684,256.510000,259.129684,262.215591,260.273924,...,244.811249,238.003970,236.347930,231.598759,225.890198,232.866664,227.337243,230.975753,234.441096,239.858583
44666,2022,50,ADP,Information Technology,Data Processing & Outsourced Services,256.510000,243.222000,256.510000,259.129684,262.215591,...,252.106184,244.811249,238.003970,236.347930,231.598759,225.890198,232.866664,227.337243,230.975753,234.441096
44667,2022,51,ADP,Information Technology,Data Processing & Outsourced Services,243.222000,239.590000,243.222000,256.510000,259.129684,...,260.273924,252.106184,244.811249,238.003970,236.347930,231.598759,225.890198,232.866664,227.337243,230.975753


In [9]:
for model_strat in model_strats:
    modeler_class = modelstratfact.build_modeler_strat(model_strat)
    speculation.db.connect()
    for year in range(start_year,end_year):
        training_set = speculation.training_set(data,year)
        prediction_set = speculation.prediction_set(data,year)
        stuff = modeler_class.recommend_model(training_set,speculation.factors)
        stuff["model"] = [pickle.dumps(x) for x in stuff["model"]]
        stuff["year"] = year
        speculation.db.store(f"{modeler_class.name}_models",stuff)
    speculation.db.disconnect()